In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Chrome 드라이버 설정 및 URL 설정
url = 'https://www.betman.co.kr/main/mainPage/gamebuy/gameSlip.do?gmId=G101&gmTs=240129'
driver = webdriver.Chrome()

try:
    # 페이지 열기
    driver.get(url)

    # 요소가 나타날 때까지 기다리기 (최대 10초)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#tbd_gmBuySlipList tr'))
    )

    # 모든 <tr> 태그 가져오기 (tbody 안의 tr 태그들)
    rows = driver.find_elements(By.CSS_SELECTOR, '#tbd_gmBuySlipList tr')

    # 결과 저장용 리스트
    game_data = []

    # 각 행에서 정보 추출
    for index, row in enumerate(rows):
        try:
            game_no = row.get_attribute("data-matchseq")

            try:
                    game_type = row.find_element(By.CSS_SELECTOR, 'span.badge.gray').text  # "일반"만 가져옴
            except Exception:
                    continue

            if "일반" in game_type:  # "일반 경기"만 필터링
        
                    # 숫자와 소수점만 남기고 나머지 제거하는 함수
                    def extract_only_numbers(text):
                        return ''.join([char for char in text if char.isdigit() or char == '.'])

                    # 승, 무, 패 배당률 추출
                    win_odds_text = row.find_element(By.CSS_SELECTOR, 'button[data-selkey="1"] span.db').text
                    win_odds = extract_only_numbers(win_odds_text).strip()

                    try:
                        draw_odds_text = row.find_element(By.CSS_SELECTOR, 'button[data-selkey="2"].btnChk span.db').text
                        draw_odds = extract_only_numbers(draw_odds_text).strip()
                    except:
                        draw_odds = 0

                    lose_odds_text = row.find_element(By.CSS_SELECTOR, 'button[data-selkey="3"] span.db').text
                    lose_odds = extract_only_numbers(lose_odds_text).strip()

                    # 간소화된 데이터 저장
                    game_data.append({
                        "game_no" : game_no,
                        "win_odds": win_odds,
                        "draw_odds": draw_odds,
                        "lose_odds": lose_odds
                    })
        except Exception as e:
            print(f"Error extracting data from row {index + 1}: {e}")

    # 데이터프레임 생성
    df = pd.DataFrame(game_data)

    # 엑셀 파일로 저장
    df.to_excel("trainingData.xlsx", index=False)
    print("엑셀 파일이 성공적으로 저장되었습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

finally:
    # 브라우저 종료
    driver.quit()

엑셀 파일이 성공적으로 저장되었습니다.


In [1]:
pip install pandas scikit-learn openpyxl

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from collections import Counter

# 훈련 데이터 불러오기
train_df = pd.read_excel("/Users/sangwoo/Desktop/pythonworkplace/python_webcrwaling/02.selenium/game_results.xlsx")

# 테스트 데이터 불러오기
test_df = pd.read_excel("/Users/sangwoo/Desktop/pythonworkplace/python_webcrwaling/trainingData.xlsx")

# game_no를 int형으로 변환
test_df["game_no"] = test_df["game_no"].astype(int)

# 데이터 전처리 - z-score 표준화 적용
train_df[["win_odds", "draw_odds", "lose_odds"]] = train_df[["win_odds", "draw_odds", "lose_odds"]].apply(zscore)
test_df[["win_odds", "draw_odds", "lose_odds"]] = test_df[["win_odds", "draw_odds", "lose_odds"]].apply(zscore)

# 훈련 데이터와 레이블 분리
X_train = train_df[["win_odds", "draw_odds", "lose_odds"]]
y_train = train_df["result"]

# KNN 모델 설정
k = 3  # 원하는 k 값 설정
knn = KNeighborsClassifier(n_neighbors=k, metric="cosine")

# 모델 훈련
knn.fit(X_train, y_train)

# 테스트 데이터에 대해 예측 수행
for i in range(len(test_df)):
    new_data = test_df.iloc[[i]][["win_odds", "draw_odds", "lose_odds"]]
    game_no = test_df.iloc[i]["game_no"]  # game_no 추출
    
    # 예측 결과와 각 클래스의 개수 확인
    distances, indices = knn.kneighbors(new_data)  # 거리와 인덱스 반환
    nearest_neighbors = y_train.iloc[indices[0]]  # k개의 가장 가까운 이웃들의 결과

    # 각 클래스(승, 무, 패)의 개수 카운트
    class_counts = Counter(nearest_neighbors)
    predicted_result = knn.predict(new_data)[0]

    # 결과 출력
    print(f"게임 번호: {game_no}")
    print(f"예측된 경기 결과: {predicted_result}")
    print(f"가장 가까운 {k}개의 이웃들에 대한 클래스 개수: {class_counts}")
    print()

게임 번호: 378.0
예측된 경기 결과: -1
가장 가까운 3개의 이웃들에 대한 클래스 개수: Counter({-1: 3})

게임 번호: 382.0
예측된 경기 결과: -1
가장 가까운 3개의 이웃들에 대한 클래스 개수: Counter({-1: 3})

게임 번호: 386.0
예측된 경기 결과: -1
가장 가까운 3개의 이웃들에 대한 클래스 개수: Counter({-1: 3})

게임 번호: 300.0
예측된 경기 결과: -1
가장 가까운 3개의 이웃들에 대한 클래스 개수: Counter({-1: 3})

게임 번호: 390.0
예측된 경기 결과: -1
가장 가까운 3개의 이웃들에 대한 클래스 개수: Counter({-1: 3})

게임 번호: 394.0
예측된 경기 결과: -1
가장 가까운 3개의 이웃들에 대한 클래스 개수: Counter({-1: 3})

게임 번호: 398.0
예측된 경기 결과: -1
가장 가까운 3개의 이웃들에 대한 클래스 개수: Counter({-1: 3})

게임 번호: 272.0
예측된 경기 결과: 1
가장 가까운 3개의 이웃들에 대한 클래스 개수: Counter({1: 3})

게임 번호: 402.0
예측된 경기 결과: 1
가장 가까운 3개의 이웃들에 대한 클래스 개수: Counter({1: 2, -1: 1})

게임 번호: 405.0
예측된 경기 결과: 1
가장 가까운 3개의 이웃들에 대한 클래스 개수: Counter({1: 2, -1: 1})

게임 번호: 409.0
예측된 경기 결과: 1
가장 가까운 3개의 이웃들에 대한 클래스 개수: Counter({1: 2, -1: 1})

게임 번호: 413.0
예측된 경기 결과: 1
가장 가까운 3개의 이웃들에 대한 클래스 개수: Counter({1: 2, -1: 1})

게임 번호: 416.0
예측된 경기 결과: -1
가장 가까운 3개의 이웃들에 대한 클래스 개수: Counter({-1: 3})

게임 번호: 420.0
예측된 경기 결과: 1
가장 가까운 3개의 이웃들에 대한 클

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Chrome 드라이버 설정 및 URL 설정
url = 'https://www.betman.co.kr/main/mainPage/gamebuy/gameSlip.do?gmId=G101&gmTs=240129'
driver = webdriver.Chrome()

try:
    # 페이지 열기
    driver.get(url)

    # 요소가 나타날 때까지 기다리기 (최대 10초)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#tbd_gmBuySlipList tr'))
    )

    # 모든 <tr> 태그 가져오기 (tbody 안의 tr 태그들)
    rows = driver.find_elements(By.CSS_SELECTOR, '#tbd_gmBuySlipList tr')

    # 결과 저장용 리스트
    game_data = []

    # 각 행에서 정보 추출
    for index, row in enumerate(rows):
        try:
            game_no = row.get_attribute("data-matchseq")

            try:
                    game_type = row.find_element(By.CSS_SELECTOR, 'span.badge.gray').text  # "일반"만 가져옴
            except Exception:
                    continue

            if "일반" in game_type:  # "일반 경기"만 필터링
        
                    # 숫자와 소수점만 남기고 나머지 제거하는 함수
                    def extract_only_numbers(text):
                        return ''.join([char for char in text if char.isdigit() or char == '.'])

                    # 승, 무, 패 배당률 추출
                    win_odds_text = row.find_element(By.CSS_SELECTOR, 'button[data-selkey="1"] span.db').text
                    win_odds = extract_only_numbers(win_odds_text).strip()

                    try:
                        draw_odds_text = row.find_element(By.CSS_SELECTOR, 'button[data-selkey="2"].btnChk span.db').text
                        draw_odds = extract_only_numbers(draw_odds_text).strip()
                    except:
                        draw_odds = 0

                    lose_odds_text = row.find_element(By.CSS_SELECTOR, 'button[data-selkey="3"] span.db').text
                    lose_odds = extract_only_numbers(lose_odds_text).strip()

                    # 간소화된 데이터 저장
                    game_data.append({
                        "game_no" : game_no,
                        "win_odds": win_odds,
                        "draw_odds": draw_odds,
                        "lose_odds": lose_odds
                    })
        except Exception as e:
            print(f"Error extracting data from row {index + 1}: {e}")

    # 데이터프레임 생성
    df = pd.DataFrame(game_data)

    # 엑셀 파일로 저장
    df.to_excel("trainingData.xlsx", index=False)
    print("엑셀 파일이 성공적으로 저장되었습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

finally:
    # 브라우저 종료
    driver.quit()

엑셀 파일이 성공적으로 저장되었습니다.
